In [140]:
import pandas as pd

## Cargamos Dataset y visualizamos las primeras 5 filas

In [141]:
db = pd.read_csv("train.csv")
print("Dimensionalidad:",db.shape)
db.head()

Dimensionalidad: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Tratamiento previo del Dataset


In [142]:
# Eliminar columna PassengerId
db.drop("PassengerId",axis=1,inplace=True)

In [143]:
# Mirar % de datos nulos por columna
for col in db.columns:
    print(col,":",db[col].isna().sum()/db.shape[0]*100,"%")

Survived : 0.0 %
Pclass : 0.0 %
Name : 0.0 %
Sex : 0.0 %
Age : 19.865319865319865 %
SibSp : 0.0 %
Parch : 0.0 %
Ticket : 0.0 %
Fare : 0.0 %
Cabin : 77.10437710437711 %
Embarked : 0.22446689113355783 %


In [144]:
# Cabina no nula y Pclass = 1
print("No nulos con 'Pclass' = 1:",db[(db["Cabin"].notnull()) & (db["Pclass"]==1)].shape[0]/db[db["Cabin"].notnull()].shape[0] * 100,"%")

# Eliminamos la columna Cabin
db.drop("Cabin",axis=1,inplace=True)


No nulos con 'Pclass' = 1: 86.27450980392157 %


In [145]:
# tabla donde la edad es nula
db[db["Age"].isna()]


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
5,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,Q
17,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,S
19,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,C
26,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,C
28,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,Q
...,...,...,...,...,...,...,...,...,...,...
859,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,C
863,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,S
868,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,S
878,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,S


In [146]:
# Imprimimos las filas de Embarked que tienen valores nulos
db[db["Embarked"].isna()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
61,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,NaN
829,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,NaN


https://www.encyclopedia-titanica.org/titanic-survivor/amelia-icard.html

https://www.encyclopedia-titanica.org/titanic-survivor/martha-evelyn-stone.html

Criada de, por ello el mismo ticket. Al ser supervivientes hay fácil acceso a información.

In [147]:
#Rellanamos los valores nulos de Embarked con S de Southampton
db["Embarked"].fillna("S",inplace=True)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked


In [148]:
# Mirar % de datos nulos por columna
for col in db.columns:
    print(col,":",db[col].isnull().sum()/db.shape[0]*100,"%")

Survived : 0.0 %
Pclass : 0.0 %
Name : 0.0 %
Sex : 0.0 %
Age : 19.865319865319865 %
SibSp : 0.0 %
Parch : 0.0 %
Ticket : 0.0 %
Fare : 0.0 %
Embarked : 0.0 %


In [150]:
# crear dataframe con las variables name y ticket
dbAux = db[["Name","Ticket"]]

# calcular % de duplicados por cada columna
for col in dbAux.columns:
    print(col,":",dbAux[col].duplicated().sum()/dbAux.shape[0]*100,"%")

Name : 0.0 %
Ticket : 23.56902356902357 %


In [154]:
# Mostrar filas con la columna de ticket igual
db[db["Ticket"]=="CA. 2343"]


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
13,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.275,S
119,0,3,"Andersson, Miss. Ellis Anna Maria",female,2.0,4,2,347082,31.275,S
541,0,3,"Andersson, Miss. Ingeborg Constanzia",female,9.0,4,2,347082,31.275,S
542,0,3,"Andersson, Miss. Sigrid Elisabeth",female,11.0,4,2,347082,31.275,S
610,0,3,"Andersson, Mrs. Anders Johan (Alfrida Konstant...",female,39.0,1,5,347082,31.275,S
813,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.275,S
850,0,3,"Andersson, Master. Sigvard Harald Elias",male,4.0,4,2,347082,31.275,S


In [155]:
db[db["Ticket"]=="347082"]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
13,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.275,S
119,0,3,"Andersson, Miss. Ellis Anna Maria",female,2.0,4,2,347082,31.275,S
541,0,3,"Andersson, Miss. Ingeborg Constanzia",female,9.0,4,2,347082,31.275,S
542,0,3,"Andersson, Miss. Sigrid Elisabeth",female,11.0,4,2,347082,31.275,S
610,0,3,"Andersson, Mrs. Anders Johan (Alfrida Konstant...",female,39.0,1,5,347082,31.275,S
813,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.275,S
850,0,3,"Andersson, Master. Sigvard Harald Elias",male,4.0,4,2,347082,31.275,S
